In [23]:
import pandas as pd
import numpy as np
import os

maggie_dat = pd.read_csv("../data/zooni/sud-project-domain-expert-3-classifications.csv")
bob_dat = pd.read_csv("../data/zooni/sud-project-domain-expert-1-classifications.csv")
# rename subject_ids
maggie_dat.rename(columns={"subject_ids":"subject_id"}, inplace=True)
bob_dat.rename(columns={"subject_ids":"subject_id"}, inplace=True)

In [24]:
# subject dat
maggie_subject_dat = pd.read_csv("../data/zooni/sud-project-domain-expert-3-subjects.csv")
bob_subject_dat = pd.read_csv("../data/zooni/sud-project-domain-expert-1-subjects.csv")

In [ ]:
# join text to subject id
bob_dat = bob_dat.merge(bob_subject_dat[['subject_id','locations']], on="subject_id")
maggie_dat = maggie_dat.merge(maggie_subject_dat[['subject_id','locations']], on="subject_id")

In [27]:
# get url from location - {"0":"https://panoptes-uploads.zooniverse.org/subject_location/d575ddca-4811-4f09-8efd-d94bd8d7c30d.txt"} 
maggie_dat['url'] = maggie_dat['locations'].apply(lambda x: x.split('"')[3])
bob_dat['url'] = bob_dat['locations'].apply(lambda x: x.split('"')[3])

In [31]:
# pull text from location url
import requests
for i in range(len(bob_dat)):
    x = requests.get(bob_dat['url'].iloc[i])
    bob_dat['text'].iloc[i] = x.text

/var/folders/jj/bt0ykz9n5_16z2gvbybl0tbc0000gn/T/ipykernel_30716/1921217895.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bob_dat['text'].iloc[i] = x.text


In [38]:
range(9,len(maggie_dat))

range(9, 101)

In [40]:
maggie_dat['text'] = np.nan
for i in range(len(maggie_dat)):
    x = requests.get(maggie_dat['url'].iloc[i])
    maggie_dat['text'].iloc[i] = x.text

/var/folders/jj/bt0ykz9n5_16z2gvbybl0tbc0000gn/T/ipykernel_30716/2920141811.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maggie_dat['text'].iloc[i] = x.text


In [32]:
# extract
def extract_annotations(df):
    df['connection'] = df['annotations'].apply(lambda x: eval(x)[0]['value'])
    df['subject'] = df['annotations'].apply(lambda x: eval(x)[1]['value'])
    df['objective'] = df['annotations'].apply(lambda x: eval(x)[2]['value'])
    return df


In [33]:
maggie_dat = extract_annotations(maggie_dat)
bob_dat = extract_annotations(bob_dat)

In [41]:
# filter midnite club
maggie_dat = maggie_dat[maggie_dat['user_name'] != "midniteclub"]
bob_dat = bob_dat[bob_dat['user_name'] != "midniteclub"]
# join annotations on text
annotations = maggie_dat[['text','connection','subject','objective']].merge(bob_dat[['text','connection','subject','objective']], on="text", suffixes=('_maggie','_bob'))

In [42]:
# get aggreement
import sklearn.metrics as metrics

# connection cohen kappa
maggie_connection = annotations['connection_maggie'].values
bob_connection = annotations['connection_bob'].values
cohen_kappa_connection = metrics.cohen_kappa_score(maggie_connection, bob_connection)

In [43]:
# print
print(f"Connection Cohen Kappa: {cohen_kappa_connection}")

Connection Cohen Kappa: 0.7661444401987008


In [44]:
maggie_subjet = annotations['subject_maggie'].values
bob_subject = annotations['subject_bob'].values
cohen_kappa_subject = metrics.cohen_kappa_score(maggie_subjet, bob_subject)

# print
print(f"Subject Cohen Kappa: {cohen_kappa_subject}")

Subject Cohen Kappa: 0.28593652769135036
